In [1]:
import numpy as np
import pandas as pd
import zipfile
from IPython.display import display, HTML

# Processing votes

In [2]:
def interpret_position(positions):
    r = positions.copy()
    r = r.apply(lambda x : 1 if x == "Yes" else ( -1 if x == "No" else 0 ))
    
    return r

USEFUL_COLUMNS = ['id','bill.number','congress','member_id','question','position', 'timestamp', 'bill.bill_id']

def process_vote_file(file_path, since_congress=115):
    print("Processing vote data for {file}".format(file=file_path), end='\r')
    ifile = ZIPPED_VOTES.open(file_path)
    
    # Read data
    df = pd.read_csv(ifile)
    
    # We drop entries without bill number as they lead to invalid bill IDs
    df = df[df['bill.number'].notna()]
    
    # We drop entries without roll call value
    df = df[df['roll_call'].notna()]
    
    # As there can be several votes per bill we need to build a unique ID
    df['id'] = df['bill.bill_id'].map(str) + "-" + df['roll_call'].map(str) + "-" + df['session'].map(str)

    # Some bill IDs and roll call numbers are poorly filled, leading to duplicates
    df = df.drop_duplicates('id')
    
    # Build timestamps
    df['timestamp'] = pd.to_datetime(df.date.map(str) + " " + df.time.map(str))
    
    # Keep only useful columns
    df = df[USEFUL_COLUMNS]
    
    # Keep only relevant congresses
    df = df[df['congress'] >= since_congress]
    
    # Convert position to numeric
    df['position'] = interpret_position(df['position'])
    
    return df

NOMINATION_VOTE = "On the Nomination"
MOTION_VOTE = "On the Motion"
AMENDMENT_VOTE = "On the Amendment"
MOTION_TO_TABLE_VOTE = "On the Motion to Table"
CLOTURE_MOTION_VOTE = "On the Cloture Motion"
BILL_PASSAGE = "On Passage of the Bill"

def build_vote_matrix(questions, data):
    """
    Given a list of questions, build a vote matrix.
    Index are senators IDs, columns are vote IDs and values are the respective positions
    of the senators with respect to the question (Yes, No, blank or NaN if the senator
    did not take part in the vote)
    
    Data should contain the following columns:
        - senators ID (member_id)
        - vote ID (id)
        - position (position)
    """    
    
    vote_matrix = data 
    
    if len(questions) > 0:
        vote_matrix = data[data['question'].isin(questions)]
        
    vote_matrix = vote_matrix.pivot(columns='id', index='member_id', values='position')
    
    return vote_matrix

## Read and compile votes per senators in a matrix

In [3]:
ZIPPED_VOTES = zipfile.ZipFile('data/votes/votes.zip')
vote_data_files = [x.filename for x in ZIPPED_VOTES.infolist() if x.filename.startswith('votes_')]

df_list = []

for filepath in vote_data_files:
    vote_data = process_vote_file(filepath)
    df_list.append(vote_data)
    
raw_votes = pd.concat(df_list)
raw_votes.to_pickle("data/processed/processed_vote_data.pickle")
raw_votes

,id,bill.number,congress,member_id,question,position,timestamp,bill.bill_id
0,hr695-115-274-2,H.R.695,115,A000360,On the Motion to Proceed,1,2018-12-21 12:31:00,hr695-115
1,hr695-115-273-2,H.R.695,115,A000360,On the Motion,-1,2018-12-19 21:30:00,hr695-115
2,pn2209-115-272-2,PN2209,115,A000360,On the Cloture Motion,1,2018-12-19 11:22:00,pn2209-115
3,s756-115-271-2,S.756,115,A000360,On the Motion,1,2018-12-18 20:22:00,s756-115
4,s756-115-270-2,S. 756,115,A000360,On the Amendment,-1,2018-12-18 20:02:00,s756-115
5,s756-115-269-2,S. 756,115,A000360,On the Amendment,-1,2018-12-18 19:43:00,s756-115
6,s756-115-268-2,S. 756,115,A000360,On the Amendment,-1,2018-12-18 19:01:00,s756-115
7,s756-115-267-2,S.756,115,A000360,On the Cloture Motion,0,2018-12-17 17:22:00,s756-115
8,sjres54-115-266-2,S.J.Res.54,115,A000360,On the Joint Resolution,-1,2018-12-13 15:03:00,sjres54-115
9,sjres54-115-265-2,S.J.Res. 54,115,A000360,On the Amendment,1,2018-12-13 14:45:00,sjres54-115


In [11]:
print(raw_votes.groupby('member_id').count())

            id  bill.number  congress  question  position  timestamp  \
member_id                                                              
A000360    599          599       599       599       599        599   
B000575    599          599       599       599       599        599   
B000944    599          599       599       599       599        599   
B001135    599          599       599       599       599        599   
B001230    599          599       599       599       599        599   
B001236    599          599       599       599       599        599   
B001261    599          599       599       599       599        599   
B001267    599          599       599       599       599        599   
B001277    599          599       599       599       599        599   
B001288    599          599       599       599       599        599   
C000127    599          599       599       599       599        599   
C000141    599          599       599       599       599       

In [13]:
vote_matrix = build_vote_matrix([BILL_PASSAGE, MOTION_VOTE, NOMINATION_VOTE], raw_votes).fillna(0)
display(vote_matrix)
vote_matrix.to_pickle("data/processed/processed_votes.pickle")
np.save("data/processed/processed_senators_ids.npy", vote_matrix.index.values)

id,hconres71-115-224-1,hconres71-115-227-1,hconres71-115-231-1,hconres71-115-234-1,hjres38-115-43-1,hr1-115-285-1,hr1-115-286-1,hr1-115-287-1,hr1-115-288-1,hr1-115-289-1,...,sconres3-115-22-1,sconres3-115-23-1,sconres3-115-24-1,sconres3-115-25-1,sconres3-115-4-1,sconres3-115-5-1,sconres3-115-6-1,sconres3-115-7-1,sconres3-115-8-1,sconres3-115-9-1
member_id,,,,,,,,,,,,,,,,,,,,,
A000360,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0
B000575,0.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1.0,-1.0,1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,1.0
B000944,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,...,-1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,1.0,-1.0
B001135,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0
B001230,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,...,-1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,1.0,-1.0
B001236,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0
B001261,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0
B001267,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,...,-1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,1.0,-1.0
B001277,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,...,-1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,1.0,-1.0


# Processing bills

In [6]:
bill_ids = pd.DataFrame(raw_votes['bill.bill_id'].drop_duplicates()).set_index('bill.bill_id')

df_bills = pd.read_csv("data/bills/active_bills.csv")
df_bills = df_bills[['bill_id','summary','summary_short']]

bill_descr = bill_ids.merge(df_bills, left_index=True, right_on='bill_id', how='left').set_index('bill_id')
bill_descr.to_csv("data/processed/processed_bill_descriptions.csv")

# Processing senators

In [13]:
pd.read_csv("data/senate_members/senate_members_115.csv")[['id','first_name', 'last_name','party','votes_with_party_pct']]

,id,first_name,last_name,party,votes_with_party_pct
0,A000360,Lamar,Alexander,R,96.91
1,B001230,Tammy,Baldwin,D,94.09
2,B001261,John,Barrasso,R,96.69
3,B001267,Michael,Bennet,D,90.17
4,B001277,Richard,Blumenthal,D,90.39
5,B000575,Roy,Blunt,R,98.12
6,B001288,Cory,Booker,D,83.75
7,B001236,John,Boozman,R,98.15
8,B000944,Sherrod,Brown,D,93.36
9,B001135,Richard,Burr,R,97.88


In [14]:
raw_senators = pd.read_csv("data/senate_members/senate_members_115.csv")[['id', 'party', 'first_name', 'last_name','votes_with_party_pct']].set_index('id')
raw_senators['name'] = raw_senators['first_name'].map(str) + " " + raw_senators['last_name'].map(str)
senators = raw_senators.loc[vote_matrix.index]
senators.to_pickle("data/processed/processed_senators.pickle")

# Saving party info
np.save("data/processed/processed_labels.npy", senators['party'].values)
labels = senators['party'].values

# Saving numerical values for party
dict_mapping = {'R':0.,'D':0.12,'I':0.223}
labels = np.vectorize(dict_mapping.get)(labels)
np.save("data/processed/processed_labels_numerical.npy", labels)